# Generate data for `test_correct_count_rate`

In [1]:
import importlib
import os
import pprint

from astropy.io import fits
import numpy as np

from jwst_magic.fsw_file_writer.buildfgssteps import BuildFGSSteps, shift_to_id_attitude

Using backend:  Qt5Agg
Your MAGIC package is up to date
Your FGS COUNTRATE package is up to date


In [2]:
def find_package_location(package):
    return importlib.util.find_spec(package).submodule_search_locations[0]


def find_repo_location(package):
    return os.path.abspath(os.path.join(find_package_location(package), os.pardir))

In [3]:
repo_location = find_repo_location(package='jwst_magic')
__location__ = os.path.join(repo_location, 'jwst_magic', 'tests')
ROOT = 'test_buildfgssteps'
SEGMENT_INFILE = os.path.join(__location__, 'data', 'unshifted_all_found_psfs_{}_G1.txt'.format(ROOT))
SELECTED_SEGS = os.path.join(__location__, 'data', '{}_regfile.txt'.format(ROOT))
CENTER_POINTING_1 = os.path.join(__location__, 'data', 'center_pointing_{}_G1.txt'.format(ROOT))
TEST_DIRECTORY = os.path.join(__location__, 'out', ROOT)

In [4]:
with fits.open( os.path.join(__location__, 'data', 'fgs_data_2_cmimf.fits')) as hdulist:
    open_image = hdulist[1].data

In [5]:
d = {}
for guider in [1, 2]:
    d[guider] = {}
    for step in ['LOSTRK', 'CAL', 'ID', 'ACQ1', 'ACQ2', 'TRK']:
        d[guider][step] = {}
        
        bias = []
        norm = []
        strips = []
        cds = []
        product = []
        
        if step != 'LOSTRK':
            n = 10
        else:
            n = 1
            
        for i in range(n):
            # Run the code
            fgs_im, guiding_selections_file, psf_center_file = shift_to_id_attitude(
                    open_image, ROOT, guider, TEST_DIRECTORY, guiding_selections_file=SELECTED_SEGS,
                    all_found_psfs_file=SEGMENT_INFILE, center_pointing_file=CENTER_POINTING_1,
            psf_center_file=None, crowded_field=False, logger_passed=True)
            BFS = BuildFGSSteps(fgs_im, guider, ROOT, step, guiding_selections_file=guiding_selections_file,
                                out_dir=TEST_DIRECTORY, shift_id_attitude=True)

            # Don't know how to test bias - since it is so variable
            if step is not 'LOSTRK':
                bias.append([np.min(BFS.bias), np.max(BFS.bias)])

            norm.append([np.min(BFS.time_normed_im), np.max(BFS.time_normed_im)])
            if step is 'ID':
                strips.append([np.min(BFS.strips), np.max(BFS.strips)])
            if 'TRK' not in step:
                cds.append([np.min(BFS.cds), np.max(BFS.cds)])
            product.append([np.min(BFS.image), np.max(BFS.image)])
        
        print()
        print(guider, step)
        print()
        for l, name in zip([bias, norm , strips , cds , product, BFS.countrate], 
                              ['bias', 'norm' , 'strips' , 'cds' , step, 'countrate']):
            if (l != [] and step == 'LOSTRK') or (name in ['countrate', 'norm']):
                d[guider][step][name] = l[0]
            elif l != [] and name != 'countrate':
                d[guider][step][name] = [round(np.average([m[0] for m in l]), -1), 
                                         round(np.average([m[1] for m in l]), -1)]

        

2021-06-02 16:27:48,843 - INFO - Started logging to file /Users/kbrooks/git_repos/jwst-magic-fork/jwst_magic/logs/2021_06_02_Wed_162748_buildfgssteps_test_buildfgssteps.log
2021-06-02 16:27:48,845 - INFO - FSW File Writing: Using /Users/kbrooks/git_repos/jwst-magic-fork/jwst_magic/tests/out/test_buildfgssteps/shifted_guiding_selections_test_buildfgssteps_G1.txt as the guiding selections file
2021-06-02 16:27:48,850 - INFO - FSW File Writing: Creating LOSTRK FSW files

1 LOSTRK

2021-06-02 16:27:48,897 - INFO - FSW File Writing: Shifting guide star to ID attitude (1024, 1024)


/Users/kbrooks/miniconda2/envs/magic/lib/python3.7/site-packages/ipykernel_launcher.py:43: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.


2021-06-02 16:27:49,516 - INFO - Successfully wrote: /Users/kbrooks/git_repos/jwst-magic-fork/jwst_magic/tests/out/test_buildfgssteps/shifted_guiding_selections_test_buildfgssteps_G1.txt
2021-06-02 16:27:49,520 - INFO - Successfully wrote: /Users/kbrooks/git_repos/jwst-magic-fork/jwst_magic/tests/out/test_buildfgssteps/shifted_all_found_psfs_test_buildfgssteps_G1.txt
2021-06-02 16:27:49,526 - INFO - Successfully wrote: /Users/kbrooks/git_repos/jwst-magic-fork/jwst_magic/tests/out/test_buildfgssteps/shifted_center_pointing_test_buildfgssteps_G1.txt
2021-06-02 16:27:49,604 - INFO - Successfully wrote: /Users/kbrooks/git_repos/jwst-magic-fork/jwst_magic/tests/out/test_buildfgssteps/FGS_imgs/shifted_test_buildfgssteps_G1.fits
2021-06-02 16:27:49,612 - INFO - Started logging to file /Users/kbrooks/git_repos/jwst-magic-fork/jwst_magic/logs/2021_06_02_Wed_162749_buildfgssteps_test_buildfgssteps.log
2021-06-02 16:27:49,613 - INFO - FSW File Writing: Using /Users/kbrooks/git_repos/jwst-magic-fo

2021-06-02 16:27:58,083 - INFO - Successfully wrote: /Users/kbrooks/git_repos/jwst-magic-fork/jwst_magic/tests/out/test_buildfgssteps/shifted_all_found_psfs_test_buildfgssteps_G1.txt
2021-06-02 16:27:58,086 - INFO - Successfully wrote: /Users/kbrooks/git_repos/jwst-magic-fork/jwst_magic/tests/out/test_buildfgssteps/shifted_center_pointing_test_buildfgssteps_G1.txt
2021-06-02 16:27:58,157 - INFO - Successfully wrote: /Users/kbrooks/git_repos/jwst-magic-fork/jwst_magic/tests/out/test_buildfgssteps/FGS_imgs/shifted_test_buildfgssteps_G1.fits
2021-06-02 16:27:58,169 - INFO - Started logging to file /Users/kbrooks/git_repos/jwst-magic-fork/jwst_magic/logs/2021_06_02_Wed_162758_buildfgssteps_test_buildfgssteps.log
2021-06-02 16:27:58,171 - INFO - FSW File Writing: Using /Users/kbrooks/git_repos/jwst-magic-fork/jwst_magic/tests/out/test_buildfgssteps/shifted_guiding_selections_test_buildfgssteps_G1.txt as the guiding selections file
2021-06-02 16:27:58,916 - INFO - FSW File Writing: Adjusting

/Users/kbrooks/miniconda2/envs/magic/lib/python3.7/site-packages/ipykernel_launcher.py:43: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


...

2021-06-02 16:29:53,388 - INFO - FSW File Writing: Creating ACQ1 FSW files
2021-06-02 16:29:53,429 - INFO - FSW File Writing: Shifting guide star to ID attitude (1024, 1024)
2021-06-02 16:29:54,072 - INFO - Successfully wrote: /Users/kbrooks/git_repos/jwst-magic-fork/jwst_magic/tests/out/test_buildfgssteps/shifted_guiding_selections_test_buildfgssteps_G2.txt
2021-06-02 16:29:54,075 - INFO - Successfully wrote: /Users/kbrooks/git_repos/jwst-magic-fork/jwst_magic/tests/out/test_buildfgssteps/shifted_all_found_psfs_test_buildfgssteps_G2.txt
2021-06-02 16:29:54,079 - INFO - Successfully wrote: /Users/kbrooks/git_repos/jwst-magic-fork/jwst_magic/tests/out/test_buildfgssteps/shifted_center_pointing_test_buildfgssteps_G2.txt
2021-06-02 16:29:54,166 - INFO - Successfully wrote: /Users/kbrooks/git_repos/jwst-magic-fork/jwst_magic/tests/out/test_buildfgssteps/FGS_imgs/shifted_test_buildfgssteps_G2.fits
2021-06-02 16:29:54,174 - INFO - Started logging to file /Users/kbrooks/git_repos/jwst-magic-f

2021-06-02 16:29:58,636 - INFO - FSW File Writing: Creating ACQ1 FSW files
2021-06-02 16:29:58,683 - INFO - FSW File Writing: Shifting guide star to ID attitude (1024, 1024)
2021-06-02 16:29:59,329 - INFO - Successfully wrote: /Users/kbrooks/git_repos/jwst-magic-fork/jwst_magic/tests/out/test_buildfgssteps/shifted_guiding_selections_test_buildfgssteps_G2.txt
2021-06-02 16:29:59,331 - INFO - Successfully wrote: /Users/kbrooks/git_repos/jwst-magic-fork/jwst_magic/tests/out/test_buildfgssteps/shifted_all_found_psfs_test_buildfgssteps_G2.txt
2021-06-02 16:29:59,335 - INFO - Successfully wrote: /Users/kbrooks/git_repos/jwst-magic-fork/jwst_magic/tests/out/test_buildfgssteps/shifted_center_pointing_test_buildfgssteps_G2.txt
2021-06-02 16:29:59,413 - INFO - Successfully wrote: /Users/kbrooks/git_repos/jwst-magic-fork/jwst_magic/tests/out/test_buildfgssteps/FGS_imgs/shifted_test_buildfgssteps_G2.fits
2021-06-02 16:29:59,427 - INFO - Started logging to file /Users/kbrooks/git_repos/jwst-magic-f

2021-06-02 16:30:03,746 - INFO - FSW File Writing: Creating ACQ2 FSW files
2021-06-02 16:30:03,787 - INFO - FSW File Writing: Shifting guide star to ID attitude (1024, 1024)
2021-06-02 16:30:04,519 - INFO - Successfully wrote: /Users/kbrooks/git_repos/jwst-magic-fork/jwst_magic/tests/out/test_buildfgssteps/shifted_guiding_selections_test_buildfgssteps_G2.txt
2021-06-02 16:30:04,522 - INFO - Successfully wrote: /Users/kbrooks/git_repos/jwst-magic-fork/jwst_magic/tests/out/test_buildfgssteps/shifted_all_found_psfs_test_buildfgssteps_G2.txt
2021-06-02 16:30:04,526 - INFO - Successfully wrote: /Users/kbrooks/git_repos/jwst-magic-fork/jwst_magic/tests/out/test_buildfgssteps/shifted_center_pointing_test_buildfgssteps_G2.txt
2021-06-02 16:30:04,600 - INFO - Successfully wrote: /Users/kbrooks/git_repos/jwst-magic-fork/jwst_magic/tests/out/test_buildfgssteps/FGS_imgs/shifted_test_buildfgssteps_G2.fits
2021-06-02 16:30:04,607 - INFO - Started logging to file /Users/kbrooks/git_repos/jwst-magic-f

2021-06-02 16:30:10,149 - INFO - FSW File Writing: Creating TRK FSW files
2021-06-02 16:30:10,235 - INFO - FSW File Writing: Shifting guide star to ID attitude (1024, 1024)
2021-06-02 16:30:10,863 - INFO - Successfully wrote: /Users/kbrooks/git_repos/jwst-magic-fork/jwst_magic/tests/out/test_buildfgssteps/shifted_guiding_selections_test_buildfgssteps_G2.txt
2021-06-02 16:30:10,865 - INFO - Successfully wrote: /Users/kbrooks/git_repos/jwst-magic-fork/jwst_magic/tests/out/test_buildfgssteps/shifted_all_found_psfs_test_buildfgssteps_G2.txt
2021-06-02 16:30:10,928 - INFO - Successfully wrote: /Users/kbrooks/git_repos/jwst-magic-fork/jwst_magic/tests/out/test_buildfgssteps/shifted_center_pointing_test_buildfgssteps_G2.txt
2021-06-02 16:30:11,005 - INFO - Successfully wrote: /Users/kbrooks/git_repos/jwst-magic-fork/jwst_magic/tests/out/test_buildfgssteps/FGS_imgs/shifted_test_buildfgssteps_G2.fits
2021-06-02 16:30:11,015 - INFO - Started logging to file /Users/kbrooks/git_repos/jwst-magic-fo

2021-06-02 16:30:22,589 - INFO - FSW File Writing: Creating TRK FSW files
2021-06-02 16:30:22,682 - INFO - FSW File Writing: Shifting guide star to ID attitude (1024, 1024)
2021-06-02 16:30:23,297 - INFO - Successfully wrote: /Users/kbrooks/git_repos/jwst-magic-fork/jwst_magic/tests/out/test_buildfgssteps/shifted_guiding_selections_test_buildfgssteps_G2.txt
2021-06-02 16:30:23,302 - INFO - Successfully wrote: /Users/kbrooks/git_repos/jwst-magic-fork/jwst_magic/tests/out/test_buildfgssteps/shifted_all_found_psfs_test_buildfgssteps_G2.txt
2021-06-02 16:30:23,306 - INFO - Successfully wrote: /Users/kbrooks/git_repos/jwst-magic-fork/jwst_magic/tests/out/test_buildfgssteps/shifted_center_pointing_test_buildfgssteps_G2.txt
2021-06-02 16:30:23,378 - INFO - Successfully wrote: /Users/kbrooks/git_repos/jwst-magic-fork/jwst_magic/tests/out/test_buildfgssteps/FGS_imgs/shifted_test_buildfgssteps_G2.fits
2021-06-02 16:30:23,392 - INFO - Started logging to file /Users/kbrooks/git_repos/jwst-magic-fo

In [6]:
pprint.pprint(d)

{1: {'ACQ1': {'ACQ1': [6460.0, 26790.0],
              'bias': [6460.0, 26790.0],
              'cds': [-20.0, 4300.0],
              'countrate': 65596.0,
              'norm': [0.0039671236, 1977.3064]},
     'ACQ2': {'ACQ2': [6460.0, 16900.0],
              'bias': [6460.0, 16890.0],
              'cds': [-10.0, 650.0],
              'countrate': 65596.0,
              'norm': [0.02249858, 137.2947]},
     'CAL': {'CAL': [1210.0, 65540.0],
             'bias': [1210.0, 30300.0],
             'cds': [-20.0, 65000.0],
             'countrate': 65596.0,
             'norm': [-5.5730787e-19, 166565.97]},
     'ID': {'ID': [1210.0, 30310.0],
            'bias': [1210.0, 30310.0],
            'cds': [-20.0, 5500.0],
            'countrate': 65596.0,
            'norm': [-1.7679394e-20, 5283.9473],
            'strips': [1210.0, 30310.0]},
     'LOSTRK': {'LOSTRK': [6.730232e-05, 1.5393685],
                'countrate': 65596.0,
                'norm': [0.0060026245, 137.2947]},
     'TRK'

# Generate data for `coordinate_tranforms`

In [7]:
import inspect
from jwst_magic.utils import coordinate_transforms

In [8]:
for func in sorted(dir(coordinate_transforms)):
    if not func.startswith('_') and '2' in func:
        print(func)
        func_to_call = getattr(coordinate_transforms, func)
#         result = func_to_call()
        arg_names = inspect.getfullargspec(func_to_call)[0]
#         print(arg_names)
    
        if 'x_raw' in arg_names:
            x, y = (1743.3, 241.9)
            
        else:
            x, y = (-55.736935, 8.139518)
            
        if 'guider' in arg_names:
            for guider in range(2):
                result = func_to_call(x, y, guider + 1)
                print(result)
        elif 'detector' in arg_names:
            result = func_to_call(x, y, 'NRCA3')
            print(result)
        else:
            result = func_to_call(x, y)
            print(result)

DHAS2Raw
(928, 206)
(1156, 207)
Idl2DHAS
(55.736935, 8.139518)
Idl2Raw
(882, 1833)
(1130, 1847)
Raw2DHAS
(-53.70147089878658, -49.33239154757665)
(-53.14152903705768, 48.84634959272053)
Raw2Det
(241.9, 1743.3)
Raw2Idl
(53.70147089878658, -49.33239154757665)
(53.14152903705768, 48.84634959272053)
Raw2Tel
(-52.61179084830326, -50.49289352206927)
(-52.97901776702914, 49.02256269690031)
nrcpixel_offset_to_v2v3_offset
(-1.7436040460019002, 0.2560291898514401)


In [9]:
ANGLE_COORDS = (-55.736935, 8.139518)
coordinate_transforms.DHAS2Raw(*ANGLE_COORDS, 1)

(928, 206)